# 아이다 코스테틱

## 데이터 불러오기

In [1]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.request import urlopen
import urllib.request as req
import re
from konlpy.tag import Okt
from konlpy.corpus import kolaw
import nltk
from collections import Counter

# 데이터 전처리
from konlpy.tag import Twitter
from collections import Counter
from wordcloud import WordCloud

# 폰트 지정해주기
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family = font_name)
# 마이너스 표시
mpl.rcParams["axes.unicode_minus"] = False

In [2]:
AIDA_order = pd.read_csv("AIDA_data/AIDA_20200820.csv", encoding = "UTF8", thousands = ",")
AIDA_order

,주문번호,품목별 주문번호,배송메시지,총 주문금액,총 결제금액,상품번호,주문상품명,주문상품명(옵션포함),수량,판매가,수령인,수령인 휴대전화,수령인 우편번호,수령인 주소,수령인 상세 주소,결제구분,결제수단,발주일,배송국가
0,20200820-0000012,20200820-0000012-01,NaN,61200.0,61200.0,446,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,1,61200.0,전예진,010-5008-0724,41522,대구광역시 북구 복현로 104-11 (복현동) 복현동소라아파트,카카오2 302호,T,신용카드,2020-08-20 00:22:13,NaN
1,20200820-0000041,20200820-0000041-01,NaN,63000.0,53423.0,199,[5차리오더]머그워트 90 모공앰플토너,[5차리오더]머그워트 90 모공앰플토너(선택=쑥토너 단품(155ml)),1,32000.0,박제희,010-6754-2577,07531,서울특별시 강서구 허준로 175 (가양동),가양6단지아파트 604동 1305호,T,"적립금,선불금",2020-08-20 01:12:56,NaN
2,20200820-0000041,20200820-0000041-02,NaN,63000.0,53423.0,92,[9차리오더]갈락토미세스83.5 부스터앰플,[8차리오더]갈락토미세스83.5 부스터앰플(필수=부스터앰플 단품(33ml)),1,31000.0,박제희,010-6754-2577,07531,서울특별시 강서구 허준로 175 (가양동),가양6단지아파트 604동 1305호,T,"적립금,선불금",2020-08-20 01:12:56,NaN
3,20200820-0000051,20200820-0000051-01,NaN,5900.0,5900.0,308,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(토너&크림선택=토너&크림미니어쳐),1,0.0,정성묵,010-2435-5614,05841,서울특별시 송파구 법원로 55 (문정동),송파아이파크 b동 1401호,T,휴대폰,2020-08-20 01:24:19,NaN
4,20200820-0000051,20200820-0000051-02,NaN,5900.0,5900.0,308,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(앰플1=수분앰플라이트),1,0.0,정성묵,010-2435-5614,05841,서울특별시 송파구 법원로 55 (문정동),송파아이파크 b동 1401호,T,휴대폰,2020-08-20 01:24:19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24571,20210220-0000459,20210220-0000459-01,NaN,21300.0,21300.0,486,아이다 웰컴 키트 3종 오픈!,아이다 웰컴 키트 3종 오픈!(필수=웰컴모이스쳐),1,15900.0,권소영,010-4003-0562,43016,대구광역시 달성군 유가읍 테크노대로5길 57,대구테크노폴리스남해오네뜨2차211동602호,T,신용카드,2021-02-20 23:27:45,NaN
24572,20210220-0000459,20210220-0000459-02,NaN,21300.0,21300.0,308,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(앰플2=수분앰플라이트),1,0.0,권소영,010-4003-0562,43016,대구광역시 달성군 유가읍 테크노대로5길 57,대구테크노폴리스남해오네뜨2차211동602호,T,신용카드,2021-02-20 23:27:45,NaN
24573,20210220-0000459,20210220-0000459-03,NaN,21300.0,21300.0,308,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(앰플1=인진쑥),1,0.0,권소영,010-4003-0562,43016,대구광역시 달성군 유가읍 테크노대로5길 57,대구테크노폴리스남해오네뜨2차211동602호,T,신용카드,2021-02-20 23:27:45,NaN
24574,20210220-0000459,20210220-0000459-04,NaN,21300.0,21300.0,308,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(진정마스크팩&필링패드=진정마스크팩1매&필링...,1,0.0,권소영,010-4003-0562,43016,대구광역시 달성군 유가읍 테크노대로5길 57,대구테크노폴리스남해오네뜨2차211동602호,T,신용카드,2021-02-20 23:27:45,NaN


In [3]:
AIDA_customer = pd.read_csv("AIDA_data/AIDA_회원정보.csv", encoding = "cp949", thousands = ",")
AIDA_customer

C:\Users\ksh27\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10,12,18,21,80,81) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SMS 수신여부,SMS 최근 수신 동의 일자,e메일 수신여부,e메일 최근 수신 동의 일자,가입시간,개인인증방법,개인정보 제3자 제공 동의 여부,개인정보 제3자 제공 동의 일자,개인정보 처리 위탁 동의 여부,개인정보 처리 위탁 동의 일자,...,환불계좌정보(은행/계좌/예금주),회원 가입일,회원구분,회원등급,회원등급적용형태,회원등급코드,회원인증여부,휴대폰번호,휴면안내(대량메일) 발송일,휴면회원 해제일
0,T,2020/09/07 12:26,T,2020/09/07 12:26,8:32:40,m,F,NaN,F,NaN,...,NaN,2013-05-03,개인,BASIC(아사원),NaN,1,T,010-9192-1999,NaN,2017/03/01 14:20
1,T,2018/07/18 22:01,T,2018/07/18 22:01,23:24:27,,F,NaN,F,NaN,...,NaN,2013-05-03,개인,SILVER(아대리),NaN,12,T,010-7163-1770,NaN,2018/03/16 13:00
2,T,2020/05/14 22:06,T,2020/05/14 22:06,17:38:26,,F,NaN,F,NaN,...,NaN,2013-11-20,개인,BASIC(아사원),NaN,1,T,010-3512-3324,NaN,2018/02/03 14:30
3,T,2020/06/30 13:21,T,2020/06/30 13:21,11:29:36,m,F,NaN,F,NaN,...,NaN,2014-02-27,개인,BASIC(아사원),NaN,1,T,010-9166-4264,2019/08/07 2:00,2019/10/07 12:50
4,T,2020/04/21 9:43,T,2020/04/21 9:43,1:48:10,,F,2015/03/12 1:48,F,2015/03/12 1:48,...,NaN,2015-03-12,개인,VIP(아이사),NaN,8,T,010-4616-8653,2016/02/11 5:30,2020/04/14 11:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9414,T,2021/03/04 23:27,T,2021/03/04 23:27,23:27:37,NaN,F,2021/03/04 23:27,F,2021/03/04 23:27,...,NaN,2021-03-04,개인,BASIC(아사원),NaN,1,T,NaN,NaN,NaN
9415,T,2021/03/04 23:31,T,2021/03/04 23:31,23:31:06,NaN,F,2021/03/04 23:31,F,2021/03/04 23:31,...,NaN,2021-03-04,개인,BASIC(아사원),NaN,1,T,NaN,NaN,NaN
9416,F,2021/03/05 4:20,F,2021/03/05 4:20,4:20:06,,F,2021/03/05 4:20,F,2021/03/05 4:20,...,NaN,2021-03-05,개인,BASIC(아사원),NaN,1,T,010-9677-4726,NaN,NaN
9417,T,2021/03/05 9:26,F,2021/03/05 9:26,9:26:54,,F,2021/03/05 9:26,F,2021/03/05 9:26,...,NaN,2021-03-05,개인,BASIC(아사원),NaN,1,T,010-5850-8433,NaN,NaN


In [4]:
df_1 = pd.read_csv("AIDA_data/AIDA_20200820.csv", encoding = "UTF8", thousands = ",")
df_2 = pd.read_csv("AIDA_data/AIDA_20200221.csv", encoding = "cp949", thousands = ",")
df_3 = pd.read_csv("AIDA_data/AIDA_20190820.csv", encoding = "cp949", thousands = ",")
df_4 = pd.read_csv("AIDA_data/AIDA_20190701.csv", encoding = "cp949", thousands = ",")
df_5 = pd.read_csv("AIDA_data/AIDA_20190220.csv", encoding = "cp949", thousands = ",")
df_6 = pd.read_csv("AIDA_data/AIDA_20180701.csv", encoding = "cp949", thousands = ",")
df_7 = pd.read_csv("AIDA_data/AIDA_20180102.csv", encoding = "cp949", thousands = ",")
df_8 = pd.read_csv("AIDA_data/AIDA_20170702.csv", encoding = "cp949", thousands = ",")
df_9 = pd.read_csv("AIDA_data/AIDA_20170101.csv", encoding = "cp949", thousands = ",")
df_10 = pd.read_csv("AIDA_data/AIDA_20160704.csv", encoding = "cp949", thousands = ",")

AIDA_order = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])
AIDA_order

,주문번호,품목별 주문번호,배송메시지,총 주문금액,총 결제금액,상품번호,주문상품명,주문상품명(옵션포함),수량,판매가,수령인,수령인 휴대전화,수령인 우편번호,수령인 주소,수령인 상세 주소,결제구분,결제수단,발주일,배송국가
0,20200820-0000012,20200820-0000012-01,NaN,61200.0,61200.0,446.0,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,1,61200.0,전예진,010-5008-0724,41522,대구광역시 북구 복현로 104-11 (복현동) 복현동소라아파트,카카오2 302호,T,신용카드,2020-08-20 00:22:13,NaN
1,20200820-0000041,20200820-0000041-01,NaN,63000.0,53423.0,199.0,[5차리오더]머그워트 90 모공앰플토너,[5차리오더]머그워트 90 모공앰플토너(선택=쑥토너 단품(155ml)),1,32000.0,박제희,010-6754-2577,07531,서울특별시 강서구 허준로 175 (가양동),가양6단지아파트 604동 1305호,T,"적립금,선불금",2020-08-20 01:12:56,NaN
2,20200820-0000041,20200820-0000041-02,NaN,63000.0,53423.0,92.0,[9차리오더]갈락토미세스83.5 부스터앰플,[8차리오더]갈락토미세스83.5 부스터앰플(필수=부스터앰플 단품(33ml)),1,31000.0,박제희,010-6754-2577,07531,서울특별시 강서구 허준로 175 (가양동),가양6단지아파트 604동 1305호,T,"적립금,선불금",2020-08-20 01:12:56,NaN
3,20200820-0000051,20200820-0000051-01,NaN,5900.0,5900.0,308.0,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(토너&크림선택=토너&크림미니어쳐),1,0.0,정성묵,010-2435-5614,05841,서울특별시 송파구 법원로 55 (문정동),송파아이파크 b동 1401호,T,휴대폰,2020-08-20 01:24:19,NaN
4,20200820-0000051,20200820-0000051-02,NaN,5900.0,5900.0,308.0,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(앰플1=수분앰플라이트),1,0.0,정성묵,010-2435-5614,05841,서울특별시 송파구 법원로 55 (문정동),송파아이파크 b동 1401호,T,휴대폰,2020-08-20 01:24:19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,20161226-0000016,20161226-0000016-03,경비실에 맡겨 주세요 감사합니다 ^^,60800.0,60800.0,NaN,원더펜 컨실러 7종 택1 [구매 시 원더톡 퍼프 무료 증정],원더펜 컨실러 7종 택1 [구매 시 원더톡 퍼프 무료 증정](색상=밝히는핑크),1,16800.0,김경미,0503-6952-9189,5119,서울특별시 광진구 광나루로56길 29,현대프라임아파트 1동 2108호,T,신용카드,2016/12/26 23:38,NaN
64,20161228-0000016,20161228-0000016-01,빠른배송부탁드립니다!!!!!!!,29700.0,23929.0,NaN,원더펜 컨실러 7종 택1 [구매 시 원더톡 퍼프 무료 증정],원더펜 컨실러 7종 택1 [구매 시 원더톡 퍼프 무료 증정](색상=1.3호 크림베이지),1,16800.0,한지원,010-8821-4073,621766,경상남도 김해시 내동 홍익아파트,106-608,T,"적립금,쿠폰,신용카드",2016/12/28 12:32,NaN
65,20161228-0000016,20161228-0000016-02,빠른배송부탁드립니다!!!!!!!,29700.0,23929.0,NaN,원더 립마스크 + 립스크럽,원더 립마스크 + 립스크럽,1,12900.0,한지원,010-8821-4073,621766,경상남도 김해시 내동 홍익아파트,106-608,T,"적립금,쿠폰,신용카드",2016/12/28 12:32,NaN
66,20161228-0000025,20161228-0000025-01,NaN,29900.0,25103.0,NaN,원더아이 애교살세럼 한정판 Black Edition [회원전용 1+1],원더아이 애교살세럼 한정판 Black Edition [회원전용 1+1],1,29900.0,,NaN,,NaN,NaN,T,"적립금,쿠폰,신용카드",2016/12/28 19:48,NaN


In [5]:
AIDA_product = pd.read_csv("AIDA_data/AIDA_제품상세.csv", encoding = "euc-kr", thousands = ",")
AIDA_product

,상품코드,자체 상품코드,진열상태,판매상태,상품분류 번호,상품분류 신상품영역,상품분류 추천상품영역,상품명,영문 상품명,상품명(관리용),...,검색엔진최적화(SEO) Keywords,검색엔진최적화(SEO) 상품 이미지 Alt 텍스트,개별결제수단설정,상품배송유형 코드,메모,스토어픽업 설정,상품 전체중량(kg),HS코드,추가항목01_제조판매자,추가항목02_제조일자
0,P00000VQ,NaN,Y,Y,124|158,N|N,N|N,[LAUNCHING] 소프트 터치 페이셜 마스크,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,NaN,NaN
1,P00000VP,NaN,Y,Y,150,N,N,오희진님 개인결제창,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,NaN,NaN
2,P00000VO,NaN,Y,Y,150,N,N,김정은님 개인결제창,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,NaN,NaN
3,P00000VN,100127,Y,N,NaN,NaN,NaN,[ONE DAY] 아이다 창고 대방출!,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,아이다코스메틱,NaN
4,P00000VL,NaN,Y,Y,150,N,N,정효원님 개인결제창,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,P00000DQ,100070,Y,Y,116|124|119,N|N|N,N|N|N,[10차리오더]나이아신아마이드5.0미백앰플,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,아이다코스메틱,NaN
140,P00000DN,NaN,Y,Y,116|124|119,N|N|N,N|N|N,[9차리오더]갈락토미세스83.5 부스터앰플,NaN,[8차리오더]갈락토미세스83.5 부스터앰플,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,아이다코스메틱,NaN
141,P00000DG,100069,Y,Y,116|124|119,N|N|N,N|N|N,[15차리오더]히알루론산50수분앰플(오리지널),NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,아이다코스메틱,NaN
142,P00000DF,NaN,N,N,150,N,N,(라이브사은품) 필링패드1파우치+진정마스크팩1매+알약비누 미니,NaN,NaN,...,NaN,NaN,NaN,C,NaN,N,1.0,NaN,아이다코스메틱,2017-06-14


## 데이터 전처리

### 주문고객 데이터 전처리

In [6]:
order = pd.DataFrame({"주문상품명(옵션)" : AIDA_order["주문상품명(옵션포함)"],
                      "주문상품명" : AIDA_order["주문상품명"], "휴대전화" : AIDA_order["수령인 휴대전화"]})
order
cols = ["주문상품명", "주문상품명(옵션)", "휴대전화"]
order = pd.DataFrame(order, columns = cols)
order.head()

,주문상품명,주문상품명(옵션),휴대전화
0,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,010-5008-0724
1,[5차리오더]머그워트 90 모공앰플토너,[5차리오더]머그워트 90 모공앰플토너(선택=쑥토너 단품(155ml)),010-6754-2577
2,[9차리오더]갈락토미세스83.5 부스터앰플,[8차리오더]갈락토미세스83.5 부스터앰플(필수=부스터앰플 단품(33ml)),010-6754-2577
3,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(토너&크림선택=토너&크림미니어쳐),010-2435-5614
4,[EVENT]아이다 미니어쳐 키트 오픈!,[EVENT]아이다 미니어쳐 키트 오픈!(앰플1=수분앰플라이트),010-2435-5614


In [7]:
order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91847 entries, 0 to 67
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   주문상품명      91847 non-null  object
 1   주문상품명(옵션)  91847 non-null  object
 2   휴대전화       81424 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


### 회원 정보 전처리

In [8]:
AIDA_customer.head()

,SMS 수신여부,SMS 최근 수신 동의 일자,e메일 수신여부,e메일 최근 수신 동의 일자,가입시간,개인인증방법,개인정보 제3자 제공 동의 여부,개인정보 제3자 제공 동의 일자,개인정보 처리 위탁 동의 여부,개인정보 처리 위탁 동의 일자,...,환불계좌정보(은행/계좌/예금주),회원 가입일,회원구분,회원등급,회원등급적용형태,회원등급코드,회원인증여부,휴대폰번호,휴면안내(대량메일) 발송일,휴면회원 해제일
0,T,2020/09/07 12:26,T,2020/09/07 12:26,8:32:40,m,F,NaN,F,NaN,...,NaN,2013-05-03,개인,BASIC(아사원),NaN,1,T,010-9192-1999,NaN,2017/03/01 14:20
1,T,2018/07/18 22:01,T,2018/07/18 22:01,23:24:27,,F,NaN,F,NaN,...,NaN,2013-05-03,개인,SILVER(아대리),NaN,12,T,010-7163-1770,NaN,2018/03/16 13:00
2,T,2020/05/14 22:06,T,2020/05/14 22:06,17:38:26,,F,NaN,F,NaN,...,NaN,2013-11-20,개인,BASIC(아사원),NaN,1,T,010-3512-3324,NaN,2018/02/03 14:30
3,T,2020/06/30 13:21,T,2020/06/30 13:21,11:29:36,m,F,NaN,F,NaN,...,NaN,2014-02-27,개인,BASIC(아사원),NaN,1,T,010-9166-4264,2019/08/07 2:00,2019/10/07 12:50
4,T,2020/04/21 9:43,T,2020/04/21 9:43,1:48:10,,F,2015/03/12 1:48,F,2015/03/12 1:48,...,NaN,2015-03-12,개인,VIP(아이사),NaN,8,T,010-4616-8653,2016/02/11 5:30,2020/04/14 11:20


In [9]:
AIDA_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9419 entries, 0 to 9418
Data columns (total 82 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SMS 수신여부                9419 non-null   object 
 1   SMS 최근 수신 동의 일자         9419 non-null   object 
 2   e메일 수신여부                9419 non-null   object 
 3   e메일 최근 수신 동의 일자         9419 non-null   object 
 4   가입시간                    9419 non-null   object 
 5   개인인증방법                  6449 non-null   object 
 6   개인정보 제3자 제공 동의 여부       9415 non-null   object 
 7   개인정보 제3자 제공 동의 일자       9410 non-null   object 
 8   개인정보 처리 위탁 동의 여부        9415 non-null   object 
 9   개인정보 처리 위탁 동의 일자        9410 non-null   object 
 10  결혼기념일                   64 non-null     object 
 11  결혼여부                    9419 non-null   object 
 12  관심분야                    6 non-null      object 
 13  국가                      6842 non-null   object 
 14  국적                      0 non-null      

### 나이 데이터 정리

In [10]:
# 20이하 :0, 20-25 : 1, 25-30 : 2, 30-35 : 3, 35-40 : 4, 40-45 : 5. 45-50 : 6, 50-55 : 7, 55-60 : 8 
AIDA_customer["나이"][0]
AIDA_customer.loc[AIDA_customer["나이"]<20, "나이"] = 0
AIDA_customer.loc[(AIDA_customer["나이"]>=20) & (AIDA_customer["나이"]<25), "나이"] = 1
AIDA_customer.loc[(AIDA_customer["나이"]>=25) & (AIDA_customer["나이"]<30), "나이"] = 2
AIDA_customer.loc[(AIDA_customer["나이"]>=30) & (AIDA_customer["나이"]<35), "나이"] = 3
AIDA_customer.loc[(AIDA_customer["나이"]>=35) & (AIDA_customer["나이"]<40), "나이"] = 4
AIDA_customer.loc[(AIDA_customer["나이"]>=40) & (AIDA_customer["나이"]<45), "나이"] = 5
AIDA_customer.loc[(AIDA_customer["나이"]>=45) & (AIDA_customer["나이"]<50), "나이"] = 6
AIDA_customer.loc[(AIDA_customer["나이"]>=50) & (AIDA_customer["나이"]<55), "나이"] = 7
AIDA_customer.loc[(AIDA_customer["나이"]>=55), "나이"] = 8

In [11]:
AIDA_customer["나이"]

0       5.0
1       6.0
2       4.0
3       5.0
4       4.0
       ... 
9414    NaN
9415    NaN
9416    4.0
9417    2.0
9418    6.0
Name: 나이, Length: 9419, dtype: float64

In [12]:
customer = pd.DataFrame({"나이" : AIDA_customer["나이"],
                         "아이디" : AIDA_customer["아이디"], 
                         "성별" : AIDA_customer["성별"], "추가사항2" : AIDA_customer["추가사항2"], 
                         "추가사항3" : AIDA_customer["추가사항3"], "추가사항4" : AIDA_customer["추가사항4"],
                         "휴대폰번호" : AIDA_customer["휴대폰번호"]})

customer

,나이,아이디,성별,추가사항2,추가사항3,추가사항4,휴대폰번호
0,5.0,aldk1980,여,NaN,NaN,NaN,010-9192-1999
1,6.0,rinoa0726,NaN,NaN,NaN,NaN,010-7163-1770
2,4.0,yjy1983,NaN,모름,홍조&예민,진정앰플,010-3512-3324
3,5.0,je8218,여,복합성,"홍조&예민,기미&잡티",미백앰플,010-9166-4264
4,4.0,kehaha,여,건성,"건조함,기미&잡티",미백앰플,010-4616-8653
...,...,...,...,...,...,...,...
9414,NaN,1649905914@k,NaN,NaN,NaN,NaN,NaN
9415,NaN,1649909240@k,NaN,NaN,NaN,NaN,NaN
9416,4.0,praisehoony,남,복합성,건조함,미백앰플,010-9677-4726
9417,2.0,155562493@n,NaN,NaN,NaN,NaN,010-5850-8433


In [13]:
cols = ["아이디", "휴대폰번호", "나이", "성별", "추가사항2", "추가사항3", "추가사항4"]
customer = pd.DataFrame(customer, columns = cols)
customer.head()

,아이디,휴대폰번호,나이,성별,추가사항2,추가사항3,추가사항4
0,aldk1980,010-9192-1999,5.0,여,NaN,NaN,NaN
1,rinoa0726,010-7163-1770,6.0,NaN,NaN,NaN,NaN
2,yjy1983,010-3512-3324,4.0,NaN,모름,홍조&예민,진정앰플
3,je8218,010-9166-4264,5.0,여,복합성,"홍조&예민,기미&잡티",미백앰플
4,kehaha,010-4616-8653,4.0,여,건성,"건조함,기미&잡티",미백앰플


In [14]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9419 entries, 0 to 9418
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   아이디     9419 non-null   object 
 1   휴대폰번호   9024 non-null   object 
 2   나이      8931 non-null   float64
 3   성별      6439 non-null   object 
 4   추가사항2   5517 non-null   object 
 5   추가사항3   5502 non-null   object 
 6   추가사항4   5490 non-null   object 
dtypes: float64(1), object(6)
memory usage: 515.2+ KB


### 제품 정보 전처리

In [15]:
AIDA_product.head()
AIDA_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 89 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   상품코드                         141 non-null    object 
 1   자체 상품코드                      88 non-null     object 
 2   진열상태                         144 non-null    object 
 3   판매상태                         144 non-null    object 
 4   상품분류 번호                      120 non-null    object 
 5   상품분류 신상품영역                   117 non-null    object 
 6   상품분류 추천상품영역                  117 non-null    object 
 7   상품명                          141 non-null    object 
 8   영문 상품명                       10 non-null     object 
 9   상품명(관리용)                     12 non-null     object 
 10  공급사 상품명                      0 non-null      float64
 11  모델명                          14 non-null     object 
 12  상품 요약설명                      1 non-null      object 
 13  상품 간략설명             

In [16]:
product = pd.DataFrame({"상품명" : AIDA_product["상품명"],"상품 간략설명" : AIDA_product["상품 간략설명"], 
                         "옵션입력" : AIDA_product["옵션입력"]})

product

,상품명,상품 간략설명,옵션입력
0,[LAUNCHING] 소프트 터치 페이셜 마스크,"★런칭기념★ 3박스 구매시 진정 마스크팩 5매 증정, 1박스 구매시 진정 마스크팩 ...",필수{★35%★소프트터치 마스크 3BOX(9개입)+사은품 진정마스크팩 5매|소프트터...
1,오희진님 개인결제창,"창고대방출(호호바오일, 미백앰플, 진정마스크팩)",NaN
2,김정은님 개인결제창,AMH 두피앰플 4박스+두피앰플 낱개 3ea+세이프캡 1set,NaN
3,[ONE DAY] 아이다 창고 대방출!,"*제품 한 품목당 수량은 1개씩 구매가 가능합니다, 한 품목에 수량이 2일시 자동 ...",필수{골라담기1(제품 3개 선택)|골라담기2(제품 5개 선택)}//골라담기{진정토너...
4,정효원님 개인결제창,수분크림 1ea 적립금 사용,NaN
...,...,...,...
139,[10차리오더]나이아신아마이드5.0미백앰플,자연유래 안심미백성분의 고농축 3중미백 기능성앰플(2ml*10),필수{미백앰플 1BOX(2ml*10)|미백앰플 3BOX(2ml*30)+사은품 필링패...
140,[9차리오더]갈락토미세스83.5 부스터앰플,"모공,피지,각질케어 데일리 필링부스터(33ml)",필수{부스터앰플 단품(33ml)|부스터앰플3EA(33ml*3)+사은품 필링패드1파우...
141,[15차리오더]히알루론산50수분앰플(오리지널),"히알루론산 50%함유, 물광피부 주름 미백 기능성 수분앰플(2ml*10)",필수{수분오리지널 1BOX(2ml*10)|(5%)수분오리지널 1BOX|(15%)수분...
142,(라이브사은품) 필링패드1파우치+진정마스크팩1매+알약비누 미니,NaN,NaN


In [17]:
cols = ["상품명", "옵션입력", "상품 간략설명"]
product = pd.DataFrame(product, columns = cols)
product.head()

,상품명,옵션입력,상품 간략설명
0,[LAUNCHING] 소프트 터치 페이셜 마스크,필수{★35%★소프트터치 마스크 3BOX(9개입)+사은품 진정마스크팩 5매|소프트터...,"★런칭기념★ 3박스 구매시 진정 마스크팩 5매 증정, 1박스 구매시 진정 마스크팩 ..."
1,오희진님 개인결제창,NaN,"창고대방출(호호바오일, 미백앰플, 진정마스크팩)"
2,김정은님 개인결제창,NaN,AMH 두피앰플 4박스+두피앰플 낱개 3ea+세이프캡 1set
3,[ONE DAY] 아이다 창고 대방출!,필수{골라담기1(제품 3개 선택)|골라담기2(제품 5개 선택)}//골라담기{진정토너...,"*제품 한 품목당 수량은 1개씩 구매가 가능합니다, 한 품목에 수량이 2일시 자동 ..."
4,정효원님 개인결제창,NaN,수분크림 1ea 적립금 사용


In [18]:
# AIDA_order.reset_index(inplace = True)
# AIDA_order

## 데이터 조인

In [19]:
order_customer= pd.merge(order, customer, left_on='휴대전화', right_on='휴대폰번호', how='inner')
order_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4194274 entries, 0 to 4194273
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   주문상품명      object 
 1   주문상품명(옵션)  object 
 2   휴대전화       object 
 3   아이디        object 
 4   휴대폰번호      object 
 5   나이         float64
 6   성별         object 
 7   추가사항2      object 
 8   추가사항3      object 
 9   추가사항4      object 
dtypes: float64(1), object(9)
memory usage: 352.0+ MB


In [20]:
order_customer = pd.DataFrame({"나이" : order_customer["나이"],
                         "아이디" : order_customer["아이디"], 
                         "성별" : order_customer["성별"], "피부타입" : order_customer["추가사항2"], 
                         "피부고민" : order_customer["추가사항3"], "선호제품" : order_customer["추가사항4"],
                         "주문상품명" : order_customer["주문상품명"], "주문상품명(옵션)" : order_customer["주문상품명(옵션)"]})

order_customer

cols = ["아이디", "나이", "성별", "피부타입", "피부고민", "선호제품", "주문상품명", "주문상품명(옵션)"]
order_customer = pd.DataFrame(order_customer, columns = cols)
order_customer.head()

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션)
0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...
1,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 인생 버블토너3차 공구!,[투비리본X아이다] 인생 버블토너1차 공구!(필수=1.(15%)[SET1] 진정토너...
2,wpgml3,3.0,여,복합성,기미&잡티,미백앰플,[5차리오더]머그워트 90 모공앰플토너,[5차리오더]머그워트 90 모공앰플토너(선택=쑥토너 단품(155ml))
3,wpgml3,3.0,여,복합성,기미&잡티,미백앰플,[9차리오더]갈락토미세스83.5 부스터앰플,[8차리오더]갈락토미세스83.5 부스터앰플(필수=부스터앰플 단품(33ml))
4,wpgml3,3.0,여,복합성,기미&잡티,미백앰플,[10차리오더]나이아신아마이드5.0미백앰플,[9차리오더]나이아신아마이드5.0미백앰플(필수=미백앰플 1BOX(2ml*10))


In [21]:
order_customer_product = pd.merge(order_customer, product, left_on='주문상품명', right_on='상품명', how='inner')
order_customer_product

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),상품명,옵션입력,상품 간략설명
0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,[투비리본X아이다] 앰플&크림 2차 공구!,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,[투비리본X아이다] 앰플&크림 2차 공구!,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,[투비리본X아이다] 앰플&크림 2차 공구!,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,[투비리본X아이다] 앰플&크림 2차 공구!,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,[투비리본X아이다] 앰플&크림 2차 공구!,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
...,...,...,...,...,...,...,...,...,...,...,...
3250919,csna2002,6.0,여,복합성,"건조함,과잉피지,기미&잡티",모름,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,NaN,NaN
3250920,62379118@n,6.0,NaN,NaN,NaN,NaN,마라징어 개인결제창,마라징어 개인결제창(필수=마라징어할로윈BOX),마라징어 개인결제창,필수{마라징어할로윈BOX},"일품징어3+마라징어3(26일(월) 순차배송, 해당 주문건은 배송 전까지 일시적으로 ..."
3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),주경민 이사님 개인결제창,본품{수분앰플오리지널|수분앰플라이트|진정앰플|모공(쑥)앰플|부스터앰플|미백앰플|진정...,NaN
3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),주경민 이사님 개인결제창,본품{수분앰플오리지널|수분앰플라이트|진정앰플|모공(쑥)앰플|부스터앰플|미백앰플|진정...,NaN


In [22]:
order_customer_product = pd.DataFrame({"나이" : order_customer_product["나이"],
                         "아이디" : order_customer_product["아이디"], 
                         "성별" : order_customer_product["성별"], "피부타입" : order_customer_product["피부타입"], 
                         "피부고민" : order_customer_product["피부고민"], "선호제품" : order_customer_product["선호제품"],
                         "주문상품명" : order_customer_product["주문상품명"], "주문상품명(옵션)" : order_customer_product["주문상품명(옵션)"],
                                       "옵션입력" : order_customer_product["옵션입력"], "상품 간략설명" : order_customer_product["상품 간략설명"]})

order_customer


cols = ["아이디", "나이", "성별", "피부타입", "피부고민", "선호제품", "주문상품명", "주문상품명(옵션)", "옵션입력", "상품 간략설명"]
order_customer_product = pd.DataFrame(order_customer_product, columns = cols)
order_customer_product.head()

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명
0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!


In [23]:
order_customer_product.to_csv('order_customer_product.csv', encoding = 'utf-8-sig')
order_customer_product

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명
0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
...,...,...,...,...,...,...,...,...,...,...
3250919,csna2002,6.0,여,복합성,"건조함,과잉피지,기미&잡티",모름,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,NaN,NaN
3250920,62379118@n,6.0,NaN,NaN,NaN,NaN,마라징어 개인결제창,마라징어 개인결제창(필수=마라징어할로윈BOX),필수{마라징어할로윈BOX},"일품징어3+마라징어3(26일(월) 순차배송, 해당 주문건은 배송 전까지 일시적으로 ..."
3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품{수분앰플오리지널|수분앰플라이트|진정앰플|모공(쑥)앰플|부스터앰플|미백앰플|진정...,NaN
3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품{수분앰플오리지널|수분앰플라이트|진정앰플|모공(쑥)앰플|부스터앰플|미백앰플|진정...,NaN


## 데이터 불러오기

In [43]:
df = pd.read_csv("order_customer_product.csv", encoding = "utf-8-sig", thousands = ",")
df

C:\Users\ksh27\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,5,6,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명
0,0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
1,1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
2,2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
3,3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
4,4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX+진정앰플1BOX+수분라이트1B...,11/23 11:00AM OPEN!!!
...,...,...,...,...,...,...,...,...,...,...,...
3250919,3250919,csna2002,6.0,여,복합성,"건조함,과잉피지,기미&잡티",모름,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,NaN,NaN
3250920,3250920,62379118@n,6.0,NaN,NaN,NaN,NaN,마라징어 개인결제창,마라징어 개인결제창(필수=마라징어할로윈BOX),필수{마라징어할로윈BOX},"일품징어3+마라징어3(26일(월) 순차배송, 해당 주문건은 배송 전까지 일시적으로 ..."
3250921,3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품{수분앰플오리지널|수분앰플라이트|진정앰플|모공(쑥)앰플|부스터앰플|미백앰플|진정...,NaN
3250922,3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품{수분앰플오리지널|수분앰플라이트|진정앰플|모공(쑥)앰플|부스터앰플|미백앰플|진정...,NaN


### 데이터 인사이드

In [44]:
## order 데이터 추가
# df
# df["옵션입력"] = [addr.split("-")[0] for addr in df["옵션입력"]]
df["옵션입력"] = df["옵션입력"].astype(str)
df["옵션입력"] = [addr.replace("+", " ") for addr in df["옵션입력"]]
df["옵션입력"] = [addr.replace("|", " ") for addr in df["옵션입력"]]
df

,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명
0,0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX 진정앰플1BOX 수분라이트1B...,11/23 11:00AM OPEN!!!
1,1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX 진정앰플1BOX 수분라이트1B...,11/23 11:00AM OPEN!!!
2,2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX 진정앰플1BOX 수분라이트1B...,11/23 11:00AM OPEN!!!
3,3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX 진정앰플1BOX 수분라이트1B...,11/23 11:00AM OPEN!!!
4,4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수{1.(20%) [앰플 SET 1] 쑥앰플1BOX 진정앰플1BOX 수분라이트1B...,11/23 11:00AM OPEN!!!
...,...,...,...,...,...,...,...,...,...,...,...
3250919,3250919,csna2002,6.0,여,복합성,"건조함,과잉피지,기미&잡티",모름,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,nan,NaN
3250920,3250920,62379118@n,6.0,NaN,NaN,NaN,NaN,마라징어 개인결제창,마라징어 개인결제창(필수=마라징어할로윈BOX),필수{마라징어할로윈BOX},"일품징어3+마라징어3(26일(월) 순차배송, 해당 주문건은 배송 전까지 일시적으로 ..."
3250921,3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품{수분앰플오리지널 수분앰플라이트 진정앰플 모공(쑥)앰플 부스터앰플 미백앰플 진정...,NaN
3250922,3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품{수분앰플오리지널 수분앰플라이트 진정앰플 모공(쑥)앰플 부스터앰플 미백앰플 진정...,NaN


### 앰플 추출하기

In [45]:
# 낮쑥밤프는?
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub(' ', text)
    return result
def text_cleaning2(text):
    hangul = re.compile(' [ㄱ-ㅣ가-힣]+앰플?[ㄱ-ㅣ가-힣]+')
    result = hangul.findall(text)
    return result

In [46]:
df["옵션입력"] = df["옵션입력"].apply(lambda x : text_cleaning(x))
df.head(5)
df["앰플"] = df["옵션입력"].apply(lambda x : text_cleaning2(x))
df

,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명,앰플
0,0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3250919,3250919,csna2002,6.0,여,복합성,"건조함,과잉피지,기미&잡티",모름,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,[선착순사은품] 쑥앰플낱개1EA+진정앰플낱개1EA,,NaN,[]
3250920,3250920,62379118@n,6.0,NaN,NaN,NaN,NaN,마라징어 개인결제창,마라징어 개인결제창(필수=마라징어할로윈BOX),필수 마라징어할로윈,"일품징어3+마라징어3(26일(월) 순차배송, 해당 주문건은 배송 전까지 일시적으로 ...",[]
3250921,3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
3250922,3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


#### 앰플 데이터만 추출

In [47]:
df = df.loc[df['옵션입력'].str.contains("앰플")][:]
df


,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명,앰플
0,0,58122215@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,4,85746159@n,3.0,NaN,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3250854,3250854,1649909240@k,NaN,NaN,NaN,NaN,NaN,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)수분앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250855,3250855,rarrarra12,3.0,여,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)진정앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250921,3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
3250922,3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


#### 성별 데이터 처리하기

In [48]:
df["성별"].fillna("여", inplace = True)
df

,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명,앰플
0,0,58122215@n,3.0,여,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,1,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,2,thsk486,4.0,여,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,3,sjh3460,3.0,여,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,4,85746159@n,3.0,여,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3250854,3250854,1649909240@k,NaN,여,NaN,NaN,NaN,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)수분앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250855,3250855,rarrarra12,3.0,여,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)진정앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250921,3250921,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
3250922,3250922,gy6686,1.0,여,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


In [49]:
df.loc[df["성별"] == "여", "성별"] = 0
df.loc[df["성별"] == "남", "성별"] = 1
df

,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명,앰플
0,0,58122215@n,3.0,0,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,2,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,3,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,4,85746159@n,3.0,0,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3250854,3250854,1649909240@k,NaN,0,NaN,NaN,NaN,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)수분앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250855,3250855,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)진정앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250921,3250921,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
3250922,3250922,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


#### 나이데이터 : 평균 값 반올림해서 4로 처리

In [50]:
AIDA_customer["나이"].mean()
df["나이"].fillna(4, inplace = True)
df

,Unnamed: 0,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,주문상품명(옵션),옵션입력,상품 간략설명,앰플
0,0,58122215@n,3.0,0,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,2,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=1.(15%) [SET1] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,3,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,4,85746159@n,3.0,0,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,[투비리본X아이다] 낮쑥밤프 앰플 1차 공구!(필수=3.(30%) [SET3] 쑥앰...,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,11/23 11:00AM OPEN!!!,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3250854,3250854,1649909240@k,4.0,0,NaN,NaN,NaN,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)수분앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250855,3250855,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),BEST앰플3종(수분/미백/진정)(필수=(20%쿠폰할인)진정앰플),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,*1인당 1박스만 구매가능합니다*,"[ 수분앰플, 미백앰플, 진정앰플]"
3250921,3250921,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(미니어쳐=앰플크림미니),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
3250922,3250922,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,주경민 이사님 개인결제창(본품=진정토너),본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,NaN,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


In [51]:
df = pd.DataFrame({"나이" : df["나이"],
                         "아이디" : df["아이디"], 
                         "성별" : df["성별"], "피부타입" : df["피부타입"], 
                         "피부고민" : df["피부고민"], "선호제품" : df["선호제품"],
                         "주문상품명" : df["주문상품명"], "옵션입력" : df["옵션입력"], "앰플" : df["앰플"]})

order_customer


cols = ["아이디", "나이", "성별", "피부타입", "피부고민", "선호제품", "주문상품명",  "옵션입력", "앰플"]
df = pd.DataFrame(df, columns = cols)
df.head()

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,옵션입력,앰플
0,58122215@n,3.0,0,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,85746159@n,3.0,0,NaN,NaN,NaN,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."


#### 결측치 행 삭제

In [55]:
df2 = df.dropna()

df2

,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,옵션입력,앰플
1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
5,kaeng2da,3.0,0,복합성,좁쌀여드름,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
6,gmlwlss625,2.0,0,복합성,"과잉피지,여드름,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...
3248876,nadaseonho,3.0,0,복합성,"건조함,홍조&예민",진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[ 진정앰플울트라]
3250855,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[ 수분앰플, 미백앰플, 진정앰플]"
3250921,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
3250922,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


In [56]:
df2.reset_index(inplace = True)
del df2["index"]
df2


,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,옵션입력,앰플
0,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,kaeng2da,3.0,0,복합성,좁쌀여드름,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,gmlwlss625,2.0,0,복합성,"과잉피지,여드름,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,복합성,"건조함,홍조&예민",진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[ 진정앰플울트라]
34652,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[ 수분앰플, 미백앰플, 진정앰플]"
34653,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
34654,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


In [58]:
df2.to_csv('AIDA_CRM_20210312.csv', encoding = 'utf-8-sig')


,아이디,나이,성별,피부타입,피부고민,선호제품,주문상품명,옵션입력,앰플
0,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
1,thsk486,4.0,0,건성,"건조함,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
2,sjh3460,3.0,0,지성,여드름,진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
3,kaeng2da,3.0,0,복합성,좁쌀여드름,수분앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
4,gmlwlss625,2.0,0,복합성,"과잉피지,여드름,좁쌀여드름,홍조&예민",진정앰플,[투비리본X아이다] 앰플&크림 2차 공구!,필수 앰플 쑥앰플 진정앰플 수분라이트 앰플 쑥앰플 진...,"[ 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, 진정앰플, 쑥앰플, ..."
...,...,...,...,...,...,...,...,...,...
34651,nadaseonho,3.0,0,복합성,"건조함,홍조&예민",진정앰플,[시앙제이X아이다] 인생 버블토너 1차 공구!,필수 옵션 프로폴리스 진정토너 옵션 머그워트 클리어토너 옵션 ...,[ 진정앰플울트라]
34652,rarrarra12,3.0,0,지성,"과잉피지,여드름,좁쌀여드름",진정앰플,BEST앰플3종(수분/미백/진정),필수 쿠폰할인 수분앰플 쿠폰할인 미백앰플 쿠폰할인 진정앰플,"[ 수분앰플, 미백앰플, 진정앰플]"
34653,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"
34654,gy6686,1.0,0,건성,"건조함,홍조&예민",진정앰플,주경민 이사님 개인결제창,본품 수분앰플오리지널 수분앰플라이트 진정앰플 모공 쑥 앰플 부스터앰플 미백앰플 진정...,"[ 수분앰플오리지널, 수분앰플라이트, 진정앰플, 부스터앰플, 미백앰플]"


In [ ]:
df = pd.read_csv("AIDA_CRM_20210312.csv", encoding = "UTF8", thousands = ",")
df

In [ ]:
# 나이 결측값은 평균인 3.7을 반올림하여 4로 대체 함
# 20이하 :0, 20-25 : 1, 25-30 : 2, 30-35 : 3, 35-40 : 4, 40-45 : 5. 45-50 : 6, 50-55 : 7, 55-60 : 8 
# 성별은 여자 0, 남자 1로 결측값 대체 함